This notebook lets you install an ocaml kernel for colab. In order to get started:
- Click on "OPEN IN PLAYGROUND".
- Run the cell below (this takes multiple minutes).
- Once the cell has run you can use some [ocaml notebook](https://colab.research.google.com/drive/107_RyCzQVL9gn7ZjeuUg5xMyT5xOK9em) on the same colab instance.

This works by this first installing the necessary apt packages, then installing opam (the ocaml package manager) and building the ocaml compiler, and finally building and installing the [ocaml-jupyter kernel](https://github.com/akabe/ocaml-jupyter).
Note that when the instance gets disconnected, it gets resetted so you have to re-install the kernel.

You can use the `use_cache` field in the form below in order to store the various binaries on your google drive the first time you run this, then re-installing should be a lot faster (this will require some google drive authentication).

If your colab instance ends up in a bad state, Runtime > Reset all runtimes... should reset it.

---



In [0]:
use_cache = "none" #@param ["none", "opam-cache.tgz"] {allow-input: true}
additional_apt_packages = "none" #@param ["none"] {allow-input: true}
additional_opam_packages = "none" #@param ["none"] {allow-input: true}

import os
import subprocess

if os.path.isdir('/root/.opam'):
  raise ValueError('opam directory already exists')

def run_script(bash_script):
  print('Running the following commands:\n%s' % bash_script)
  env = dict(os.environ)
  env['OCAML_INSTALL'] = bash_script
  process = subprocess.Popen(
    '/bin/bash -c "$OCAML_INSTALL"',
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env=env)
  stdout, stderr = process.communicate()
  if stdout: print(stdout.decode())
  if stderr: print(stderr.decode())
  return_code = process.wait()
  if return_code:
    raise ValueError('subprocess failed %d' % return_code)

bash_script = []

apt_packages_ = [ 'libffi-dev', 'libgmp-dev', 'm4' ]
if additional_apt_packages != "none": apt_packages_ += additional_apt_packages.split(' ')

opam_packages_ = [ 'base', 'jupyter' ]
if additional_opam_packages != "none": opam_packages_ += additional_opam_packages.split(' ')

# Install the system packages.
run_script('apt install ' + ' '.join(apt_packages_))

read_cache_ = None
write_cache_ = None
if use_cache is not None and use_cache != 'none' and use_cache:
  from google.colab import drive
  drive.mount('/content/drive')
  cache_file = '/content/drive/My Drive/' + use_cache
  if os.path.isfile(cache_file):
    read_cache_ = cache_file
  else:
    write_cache_ = cache_file

# Install opam.
run_script(
  '(echo "" && yes Y) | (sh <(curl -sL https://raw.githubusercontent.com/ocaml/opam/master/shell/install.sh))')
  
if read_cache_ is None:
  run_script('echo N | opam init --disable-sandboxing --compiler=4.07.1')
  
  # Install opam packages.
  run_script('opam install -y ' + ' '.join(opam_packages_))
  if write_cache_ is not None:
    run_script('(cd; tar -czf "%s" .opam)' % write_cache_)
else:
  run_script('(cd; tar -xzf "%s")' % read_cache_)

run_script('echo \'#use "topfind";;\' >> /root/.ocamlinit')
run_script('jupyter kernelspec install --name ocaml-jupyter /root/.opam/default/share/jupyter')

In [0]:
# Running this cell lists all the installed jupyter kernel.
# After running the previous cell, you should have an ocaml-jupyter kernel listed.
!jupyter-kernelspec list